# Tree Based Classifier Models

In [103]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight

In [104]:
from rdkit.Chem import rdFingerprintGenerator

# Define a function to convert SMILES to fingerprints
def smiles_to_fingerprints(smiles, n_bits=1024):
    """
    Converts a SMILES string to a molecular fingerprint.

    Parameters
    ----------
    smiles : str
        A SMILES string representing the molecular structure.
    n_bits : int, Optional, default: 1024
        The number of bits in the fingerprint.

    Returns
    -------
    list
        A list of integers representing the molecular fingerprint.
        Returns None if the SMILES string is invalid.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=n_bits)
        return mfpgen.GetFingerprint(mol)
    else:
        return None # Return a zero vector if the SMILES is invalid

In [105]:
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

def return_scores(all_y_pred, all_y_true, y, title="Define Title"):

    # Get unique classes
    classes = np.unique(all_y_true)

    # Binarize the true and predicted labels for multi-class AUC calculation
    y_true_bin = label_binarize(all_y_true, classes=classes)
    y_pred_bin = label_binarize(all_y_pred, classes=classes)

    # Calculate AUC for each class using One-vs-Rest (OvR) method
  
    # Average AUC across all classes
    avg_auc = roc_auc_score(y_true_bin, y_pred_bin, average="macro", multi_class="ovr")

    # Weighted AUC, which considers the support of each class
    weighted_auc = roc_auc_score(y_true_bin, y_pred_bin, average="weighted", multi_class="ovr")


    print(f"{title}")
    # Calculate overall accuracy
    overall_accuracy = accuracy_score(all_y_true, all_y_pred)

    # Print classification report for per-class metrics
    print("\nPer-Class Classification Report:")
    print(classification_report(all_y_true, all_y_pred, target_names=np.unique(y), digits=4))

    # Calculate weighted and macro averages for precision, recall, and F1 score
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        all_y_true, all_y_pred, average='weighted'
    )

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        all_y_true, all_y_pred, average='macro'
    )

    print(f"Overall Accuracy: {overall_accuracy:.4f}")

    # Print weighted averages
    print("\nWeighted Averages:")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1 Score: {f1_weighted:.4f}")
    print(f"Weighted Average AUC: {weighted_auc:.4f}")

    # Print unweighted (macro) averages
    print("\nMacro Averages (Unweighted):")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1 Score: {f1_macro:.4f}")
    print(f"Average AUC (macro): {avg_auc:.4f}")

    # Optional: Calculate per-class precision, recall, and F1 score explicitly
    # Get per-class metrics using precision_recall_fscore_support without averaging
    precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
        all_y_true, all_y_pred, average=None, labels=np.unique(all_y_true)
    )

# XGBoost on fingerprints

In [ ]:
# Load datasets
train = pd.read_csv("fart_train.csv")
val = pd.read_csv("fart_val.csv")
test = pd.read_csv("fart_test.csv")

# Calculate fingerprints for train, val, and test sets separately
train["fingerprints"] = train["Canonicalized SMILES"].apply(smiles_to_fingerprints)
val["fingerprints"] = val["Canonicalized SMILES"].apply(smiles_to_fingerprints)
test["fingerprints"] = test["Canonicalized SMILES"].apply(smiles_to_fingerprints)

# Convert features list to a numpy array for modeling
X_train = np.array(train['fingerprints'].tolist())
y_train = train['Canonicalized Taste'].values

X_val = np.array(val['fingerprints'].tolist())
y_val = val['Canonicalized Taste'].values

X_test = np.array(test['fingerprints'].tolist())
y_test = test['Canonicalized Taste'].values

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit the encoder and transform the labels to integers
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)

[11:45:10] WARNING: not removing hydrogen atom without neighbors
[11:45:10] WARNING: not removing hydrogen atom without neighbors
[11:45:10] WARNING: not removing hydrogen atom without neighbors
[11:45:11] WARNING: not removing hydrogen atom without neighbors
[11:45:12] WARNING: not removing hydrogen atom without neighbors


In [107]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define train and validation sets (you already have these as separate files)
# X_train, y_train_encoded: training data
# X_val, y_val_encoded: validation data

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 10, 15],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8, 1.0],
}

# Initialize an empty list to store results
results = []

# Perform a grid search manually
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for learning_rate in param_grid['learning_rate']:
            for subsample in param_grid['subsample']:
                # Define the model with current hyperparameters
                xgb = XGBClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    subsample=subsample,
                    objective='multi:softprob',
                    eval_metric='mlogloss',
                    random_state=101,
                    tree_method='hist'  # Use histogram-based algorithm for efficiency
                )
                
                # Train the model on the train split
                xgb.fit(X_train, y_train_encoded)
                
                # Validate the model on the validation split
                y_val_pred = xgb.predict(X_val)
                val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
                
                # Store the results
                results.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'subsample': subsample,
                    'val_accuracy': val_accuracy
                })

# Find the best hyperparameters based on validation accuracy
best_params = max(results, key=lambda x: x['val_accuracy'])
print("Best Parameters:", best_params)

# Train the best model on the train set
best_xgb = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    objective='multi:softprob',
    eval_metric='mlogloss',
    random_state=101,
    tree_method='hist'
)

# Train the best model
best_xgb.fit(X_train, y_train_encoded)

# Test on the test set (if available)
# Get probability predictions
y_pred_proba = best_xgb.predict_proba(X_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

# Call the return_scores function
return_scores(y_pred, y_test_encoded, y_test, title="XGBoost on fingerprints")


Best Parameters: {'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.1, 'subsample': 0.6, 'val_accuracy': 0.9028393966282166}
XGBoost on fingerprints

Per-Class Classification Report:
              precision    recall  f1-score   support

      bitter     0.8528    0.7210    0.7814       233
        sour     0.8893    0.9118    0.9004       238
       sweet     0.9434    0.9511    0.9473      1473
       umami     0.6667    0.3333    0.4444         6
   undefined     0.7323    0.7829    0.7568       304

    accuracy                         0.8988      2254
   macro avg     0.8169    0.7400    0.7661      2254
weighted avg     0.8991    0.8988    0.8981      2254

Overall Accuracy: 0.8988

Weighted Averages:
Weighted Precision: 0.8991
Weighted Recall: 0.8988
Weighted F1 Score: 0.8981
Weighted Average AUC: 0.9098

Macro Averages (Unweighted):
Macro Precision: 0.8169
Macro Recall: 0.7400
Macro F1 Score: 0.7661
Average AUC (macro): 0.8520


# Balanced Random Forest

In [108]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 10, 15]
}

# Initialize an empty list to store results
results = []

# Perform a grid search manually
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        # Define the model with current hyperparameters and future behavior
        brf = BalancedRandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            criterion="gini",
            random_state=101,
            sampling_strategy='all',  # Future behavior: Use all classes equally
            replacement=True,         # Future behavior: Replace samples
            bootstrap=False            # Future behavior: Disable bootstrap sampling
        )
        
        # Train the model on the train split
        brf.fit(X_train, y_train_encoded)
        
        # Validate the model on the validation split
        y_val_pred = brf.predict(X_val)
        val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
        
        # Store the results
        results.append({
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'val_accuracy': val_accuracy
        })

# Find the best hyperparameters based on validation accuracy
best_params = max(results, key=lambda x: x['val_accuracy'])
print("Best Parameters:", best_params)

# Train the best model on the train set with future behavior
best_brf = BalancedRandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    criterion="gini",
    random_state=101,
    sampling_strategy='all',
    replacement=True,
    bootstrap=False
)

# Train the best model
best_brf.fit(X_train, y_train_encoded)

# Test on the test set
y_pred = best_brf.predict(X_test)

# Call the return_scores function
return_scores(y_pred, y_test_encoded, y_test, title="Balanced Random Forest")


Best Parameters: {'n_estimators': 100, 'max_depth': 15, 'val_accuracy': 0.790150842945874}
Balanced Random Forest

Per-Class Classification Report:
              precision    recall  f1-score   support

      bitter     0.6852    0.4764    0.5620       233
        sour     0.6145    0.8908    0.7273       238
       sweet     0.9579    0.8344    0.8919      1473
       umami     0.0909    0.6667    0.1600         6
   undefined     0.5738    0.7928    0.6657       304

    accuracy                         0.7972      2254
   macro avg     0.5845    0.7322    0.6014      2254
weighted avg     0.8393    0.7972    0.8079      2254

Overall Accuracy: 0.7972

Weighted Averages:
Weighted Precision: 0.8393
Weighted Recall: 0.7972
Weighted F1 Score: 0.8079
Weighted Average AUC: 0.8650

Macro Averages (Unweighted):
Macro Precision: 0.5845
Macro Recall: 0.7322
Macro F1 Score: 0.6014
Average AUC (macro): 0.8391


# XGBoost on fingerprints and 15 Mordred Descriptors

In [109]:
from mordred import Calculator, descriptors  # Requires Python 3.10 or earlier
from rdkit import Chem

# Create a Mordred Calculator
calc = Calculator()

# Register specific descriptors to the calculator
calc.register(descriptors.Autocorrelation.ATSC(0, 'c'))
calc.register(descriptors.Autocorrelation.ATSC(0, 'se'))
calc.register(descriptors.Autocorrelation.AATS(0, 'i'))
calc.register(descriptors.Autocorrelation.ATSC(1, 'p'))
calc.register(descriptors.Autocorrelation.AATSC(2, 'se'))
calc.register(descriptors.Autocorrelation.AATSC(0, 'm'))
calc.register(descriptors.Autocorrelation.AATSC(1, 'Z'))
calc.register(descriptors.Autocorrelation.AATSC(2, 'are'))
calc.register(descriptors.Autocorrelation.AATSC(1, 'pe'))
calc.register(descriptors.AdjacencyMatrix.AdjacencyMatrix('SpDiam'))
calc.register(descriptors.Autocorrelation.ATSC(1, 'c'))
calc.register(descriptors.Autocorrelation.ATSC(1, 'se'))
calc.register(descriptors.Autocorrelation.ATSC(1, 'Z'))
calc.register(descriptors.Autocorrelation.ATSC(1, 'm'))
calc.register(descriptors.Autocorrelation.ATSC(4, 's'))


def generate_descriptors(smiles, calculator=calc):
    """
    Calculates selected descriptors from SMILES.

    Parameters
    ----------
    smiles : str
        A SMILES string representing the molecular structure.
    calculator : Calculator
        A Mordred Calculator with specified descriptors.

    Returns
    -------
    list
        A list of descriptor values or None if calculation fails.
    """
    if calculator is None or smiles is None:
        return None

    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        return None

    try:
        return list(calculator(mol))  # Convert the Mordred Result to a list
    except Exception as error:
        return None  # Handle errors gracefully


# Apply descriptor generation separately for train, val, and test sets
train['mordred_descriptors'] = train['Canonicalized SMILES'].apply(generate_descriptors)
val['mordred_descriptors'] = val['Canonicalized SMILES'].apply(generate_descriptors)
test['mordred_descriptors'] = test['Canonicalized SMILES'].apply(generate_descriptors)

# Replace missing descriptors with zero vectors
train['mordred_descriptors'] = train['mordred_descriptors'].apply(
    lambda x: np.zeros(calc.size, dtype=np.float32) if x is None else np.array(x, dtype=np.float32)
)
val['mordred_descriptors'] = val['mordred_descriptors'].apply(
    lambda x: np.zeros(calc.size, dtype=np.float32) if x is None else np.array(x, dtype=np.float32)
)
test['mordred_descriptors'] = test['mordred_descriptors'].apply(
    lambda x: np.zeros(calc.size, dtype=np.float32) if x is None else np.array(x, dtype=np.float32)
)

[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:51] WARNING: not removing hydrogen atom without neighbors
[11:50:54] WARNING: not removing hydrogen atom without neighbors
[11:50:54] WARNING: not removing hydrogen atom without neighbors
[11:50:54] WARNING: not removing hydrogen atom without neighbors
[11:51:01] WARNING: not removing hydrogen atom without neighbors
[11:51:01] WARNING: not removing hydrogen atom without neighbors
[11:51:01] WARNING: not removing hydrogen atom without neighbors
[11:51:08] WARNING: not removing hydrogen atom without neighbors
[11:51:08] WARNING: not removing hydrogen atom without neighbors
[11:51:08] WARNING: not removing hydrogen atom without neighbors


In [110]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Ensure Mordred descriptors and fingerprints are in compatible formats
# Convert fingerprints (ExplicitBitVect) to NumPy arrays
train['fingerprints'] = train['fingerprints'].apply(lambda x: np.array(x, dtype=np.float32))
val['fingerprints'] = val['fingerprints'].apply(lambda x: np.array(x, dtype=np.float32))
test['fingerprints'] = test['fingerprints'].apply(lambda x: np.array(x, dtype=np.float32))

# Ensure Mordred descriptors are lists or arrays and fill missing descriptors with zeros
train['mordred_descriptors'] = train['mordred_descriptors'].apply(
    lambda x: np.array(x, dtype=np.float32) if x is not None else np.zeros(calc.size, dtype=np.float32)
)
val['mordred_descriptors'] = val['mordred_descriptors'].apply(
    lambda x: np.array(x, dtype=np.float32) if x is not None else np.zeros(calc.size, dtype=np.float32)
)
test['mordred_descriptors'] = test['mordred_descriptors'].apply(
    lambda x: np.array(x, dtype=np.float32) if x is not None else np.zeros(calc.size, dtype=np.float32)
)

# Combine fingerprints and descriptors for X_train
X_train = np.array([
    np.concatenate((fingerprint, descriptor))
    for fingerprint, descriptor in zip(train['fingerprints'], train['mordred_descriptors'])
])

# Combine fingerprints and descriptors for X_val
X_val = np.array([
    np.concatenate((fingerprint, descriptor))
    for fingerprint, descriptor in zip(val['fingerprints'], val['mordred_descriptors'])
])

# Combine fingerprints and descriptors for X_test
X_test = np.array([
    np.concatenate((fingerprint, descriptor))
    for fingerprint, descriptor in zip(test['fingerprints'], test['mordred_descriptors'])
])

# Extract labels for train, val, and test sets
y_train = train['Canonicalized Taste'].values
y_val = val['Canonicalized Taste'].values
y_test = test['Canonicalized Taste'].values

# Initialize the LabelEncoder and encode labels for XGBoost
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)

In [111]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 10, 15],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8, 1.0],
}

# Initialize an empty list to store results
results = []

# Perform a grid search manually
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for learning_rate in param_grid['learning_rate']:
            for subsample in param_grid['subsample']:
                # Define the model with current hyperparameters
                xgb = XGBClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    subsample=subsample,
                    objective='multi:softprob',
                    eval_metric='mlogloss',
                    random_state=101,
                    tree_method='hist'  # Use histogram-based algorithm for efficiency
                )
                
                # Train the model on the train split
                xgb.fit(X_train, y_train_encoded)
                
                # Validate the model on the validation split
                y_val_pred = xgb.predict(X_val)
                val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
                
                # Store the results
                results.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'subsample': subsample,
                    'val_accuracy': val_accuracy
                })

# Find the best hyperparameters based on validation accuracy
best_params = max(results, key=lambda x: x['val_accuracy'])
print("Best Parameters:", best_params)

# Train the best model on the train set
best_xgb = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    objective='multi:softprob',
    eval_metric='mlogloss',
    random_state=101,
    tree_method='hist'
)

# Train the best model
best_xgb.fit(X_train, y_train_encoded)

# Test on the test set (if available)
# Get probability predictions
y_pred_proba = best_xgb.predict_proba(X_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

# Call the return_scores function
return_scores(y_pred, y_test_encoded, y_test, title="XGBoost on fingerprints+descriptors")


Best Parameters: {'n_estimators': 200, 'max_depth': 15, 'learning_rate': 0.1, 'subsample': 0.8, 'val_accuracy': 0.9023957409050577}
XGBoost on fingerprints+descriptors

Per-Class Classification Report:
              precision    recall  f1-score   support

      bitter     0.8571    0.7468    0.7982       233
        sour     0.9064    0.8950    0.9006       238
       sweet     0.9394    0.9464    0.9428      1473
       umami     1.0000    0.3333    0.5000         6
   undefined     0.7182    0.7796    0.7476       304

    accuracy                         0.8962      2254
   macro avg     0.8842    0.7402    0.7779      2254
weighted avg     0.8977    0.8962    0.8959      2254

Overall Accuracy: 0.8962

Weighted Averages:
Weighted Precision: 0.8977
Weighted Recall: 0.8962
Weighted F1 Score: 0.8959
Weighted Average AUC: 0.9059

Macro Averages (Unweighted):
Macro Precision: 0.8842
Macro Recall: 0.7402
Macro F1 Score: 0.7779
Average AUC (macro): 0.8513
